# Friday Sept 28th, 2021, 2:38PM -- Gates, CMU

Reviewing old questions for practice and general mental-upkeep.

## Q1. Implement LRU cache.

* PUT: Add (k, v) pair if not exists, or update value if key exists.
* If cache at capaciy, drop the LRU (k, v) pair.
* GET: Return value if (k) is present, else -1.

Ans.

So. Whatever `key` you use. You need to push that to the front of the stack.

And, for any new `key` added, you need to move that to the front of the stack also.

OrderedDict in Python will let you do this. The harder approach would be to implement using LinkedLists. *Try both*.


In [11]:
from collections import OrderedDict

class LRUCacheOD:
    def __init__(self, size):
        self.size = size
        self.cache = OrderedDict()
        
    def __repr__(self):
        return list(self.cache.items())
    
    def __str__(self):
        return str(list(self.cache.items()))
        
    def add(self, k, v):
        if k in self.cache:
            self.cache[k] = v
            self.cache.move_to_end(k)
            
        else:
            if len(self.cache) < self.size:
                self.cache[k] = v
                self.cache.move_to_end(k)
            else:
                # You will have to delete LRU.
                todel = next(iter(self.cache))
                del self.cache[todel]
                
                self.cache[k] = v
                self.cache.move_to_end(k)
                
    def get(self, k):
        if k in self.cache:
            self.cache.move_to_end(k)
            
        return self.cache.get(k, -1)

In [13]:
t = LRUCacheOD(2)
assert t.get(5) == -1
assert t.get(1) == -1
t.add(1, 1)
assert t.get(1) == 1
t.add(2, 2)
assert t.get(2) == 2
t.add(1, 11)
print(t)
t.add(3, 3)
print(t)
assert 2 not in t.cache
assert 1 in t.cache
assert 3 in t.cache

[(2, 2), (1, 11)]
[(1, 11), (3, 3)]


In [ ]:
class Node:
    """
    Ref implementaiton for doubly linked list.
    """
    def __init__(self, k, v, before=None, after=None):
        self.k = k
        self.v = v
        self.before = before
        self.after = after

class LRUCacheLL:
    """
    LinkedList implementation.
    """
    def __init__(self, size):
        self.size = size
        self.cache = {}
        # Cache will map keys to nodes in the Doubly Linked List.
        self.lru = None
        self.head = None
        
    def add(self, k, v):
        if k in self.cache:
            # Update value first.
            # Then move to end.
            self.cache[k].v = v
            # If this was the LRU node, then you will have to update LRU
            # You will have to update head.
            
        else:
            n = Node(k, v, self.head)
            if len(self.cache) < self.size:
                self.head = n
            else:
                # Delete.
                

## Q2. List of meeting start and end times. Find the number of rooms required.

Given a list of meetings with start and end times: [(0, 30), (5, 15), (20, 50)] can you find the min. number of meeting rooms required?

Try to think of the simplest solution first:
1. You will need 1 room for the first meeting.
2. For the second, check against this room to see if it's free. If it is, update the room's `available` time, else get a new room.
3. For the third, you will have to check both rooms and make a decision.

In [27]:
class MeetingSlow:
    """
    Slow implementaiton. Iterates over all existing rooms to check.
    """
    def __init__(self):
        self.rooms = []
        
    def solve(self, splits):
        if len(splits) <= 1:
            return len(splits)
        
        _, first = splits[0]
        self.rooms.append(first)
        
        for i in range(1, len(splits)):
            start, end = splits[i]
            found = False
            for j, available in enumerate(self.rooms):
                if start >= available:
                    self.rooms[j] = end
                    found = True
                    break
                    
            if found is False:
                self.rooms.append(end)
            
            print(self.rooms)
            
        return len(self.rooms)

In [28]:
t = MeetingSlow()
t.solve([(0, 30), (5, 15), (40, 50)])

[30, 15]
[50, 15]


2

In [44]:
import heapq

class MeetingFast:
    """
    Fast version using heapq.
    """
    def __init__(self):
        self.rooms = []
    
    def solve(self, splits):
        if len(splits) <= 1:
            return 1
        
        for start, end in splits:
            if len(self.rooms) == 0:
                self.rooms.append(end)
            else:
                first = self.rooms[0]
                if start >= first:
                    heapq.heappop(self.rooms)
                    heapq.heappush(self.rooms, end)
                else:
                    heapq.heappush(self.rooms, end)
                    
        return len(self.rooms)

In [46]:
t = MeetingFast()
t.solve([(0, 30), (5, 15), (40, 50)])

2

In [35]:
for i, (start, end) in enumerate([(0, 30), (5, 15), (40, 50)]):
    print(i, start, end)

0 0 30
1 5 15
2 40 50


## Q3. Shortest path b/w two words.

Start word: `hit`

End word: `cog`

You have a word list, and you can only edit a word by changing one letter. Can you find the nunber of intermediate words between the start and end?

`hit -> hot -> cog` : Ans is 2.

In [53]:
from collections import deque

class ShortestPath:
    """
    Mutate, and apply BFS.
    Keep iterating -- prune a node if it is not in the list.
    Once you find, return the current count.
    Ensure that you DONOT REPEAT a previously seen word!!!
    """
    def __init__(self):
        pass
    
    def solve(self, start, end, words):
        words = set(words)
        if start not in words or end not in words:
            return 0
        
        q = deque([(start, 0)])
        seen = set()
        while q:
            temp, dist = q.popleft()
            if temp == end:
                return dist
            else:
                if temp not in seen:
                    seen.add(temp)
                    # Mutate.
                    for i in range(3):
                        for char in 'qwertyuiopasdfghjklzxcvbnm':
                            alt = temp[:i] + char + temp[i+1:]
                            if alt not in seen:
                                q.append((alt, dist+1))
                                
        return -1

In [54]:
t = ShortestPath()
t.solve('hit', 'cog', ['hit', 'cog', 'bog', 'mog', 'bit', 'hog'])

3

In [48]:
s.add('hello')

In [49]:
s

{'hello'}

## Q4. `GetRandom` set.

Implement the following with O(1) time:
* PUT
* GET
* DEL
* SAMPLE -- should return an element with uniform probability

**Ans**. You can just start working with a `set`. But the sampling will require a list, and the conversion will require O(n) time.

Now, you can save the elements in a list, but then deletion will be a problem.

So, you have to delete an element from the list in O(1) time -- i.e. a `pop` operation.

The solution is to swap the element you want to delete with the *last* element in the list, and then pop!

In [56]:
import random

class GetRandom:
    def __init__(self):
        self.positions = {}  # key -> location
        self.container = []  # keys
        
    def put(self, x):
        if x not in self.positions:
            self.positions[x] = len(self.container)
            self.container.append(x)
            
    def get(self, x):
        return True if x in self.positions else False
    
    def delete(self, x):
        if x in self.positions:
            if self.container[-1] == x:
                self.container.pop()
                del self.positions[x]
            else:
                last = self.container.pop()
                i = self.positions[x]
                self.positions[last] = i
                del self.positions[x]

## Q5. Find smallest positive integer missing from the array.

## Q6. Maximize Profit.

Array of prices. You can buy one day, and sell in the future. Single trade only. How to maximize profit?

**Ans**: Each day, you *could* buy the stock. The question is when in the future do you sell? The answer is the MAXIMUM price in the future.

Work your way backwards from the right end, and track the max value.

## Q7. Generate all permutations of a given string.

In [59]:
class Permute:
    def __init__(self):
        pass
    
    def solve(self, string):
        def permute(sub):
            if len(sub) == 1:
                return [sub]
            
            ans = []
            for i in range(len(sub)):
                root = sub[i]
                rest = sub[:i] + sub[i+1:]
                for temp in permute(rest):
                    ans.append(root + temp)
                    
            return ans
        
        return permute(string)

In [60]:
t = Permute()
t.solve('abc')

['abc', 'acb', 'bac', 'bca', 'cab', 'cba']

## Q8. Longest substring without repeating characters.

`s = "abcabcbb"`

Start from left to right -- standard two-pointer approach.

Maintain a dict where you track the seen characters.
* So long as you keep seeing new chars, keep upping.
* Whenever you see a repeatition, pause, and start dropping from prev till repeating char goes away.
* You do not need to track the intermediate length, because they will have to be smaller than the array you started from.

In [65]:
from collections import defaultdict

class LongestSubRepeat:
    def solve(self, string):
        i = 0
        j = 0
        seen = set()
        
        ans = 0
        
        while j < len(string):
            current = string[j]
            if current not in seen:
                seen.add(current)
                j += 1
            else:
                ans = max(ans, j-i)
                while current in seen and i < j:
                    seen.remove(string[i])
                    i += 1
                    
        ans = max(ans, j-i)
        return ans

In [68]:
t = LongestSubRepeat()
t.solve('abcabcbbabcd')

4

## Q9. Return product of array except self.

nums = [1, 2, 3, 4]

ans = [24, 12, 8, 6]

Okay. So we need products to the left, and then products to the right...

In [ ]:
class ProductSelf:
    def solve(self, array):
        zeros = 0
        for x in array:
            if x == 0:
                zeros += 1
                
        if zeros == 1:
            prod = 1
            for x in array:
                if x != 0:
                    prod *= x
                    
            return [prod if x == 0 else 0 for x in array]
        
        elif zeros > 1:
            return [0 for x in array]
        
        else:
            # No zeros now.
            